In [2]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

In [42]:
from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [66]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''

video_id = '30a5yFBd7Fo'

tran = get_transcript(video_id)

tran

res = ''
for text in tran:
#     if int(text['start']) < 1366:
    res += str(int(text['start'])) + '\n' + text['text'] 

res


"0\nuh these are some guys i saw in kyoto3\nand they're tearing down a scaffolding6\nand i just think they're9\namazingly poetic and how they do their12\nwork13\nso in a startup um16\nfounders basically have to figure out19\nhow to optimize for a relationship that21\nlasts for like22\n10 years and that's a crazy thing to do25\nwith someone you might only know for a27\ncouple of months28\nor have only known in a sort of work30\nsetting and the thing is like the only32\nmodels33\nfor understanding that kind of36\nrelationship37\nactually come probably from our parents40\nand so i'd like to start off with some42\nmarriage research43\nso this is john gottman he studied his47\nmarriages in seattle he's been featured49\nin this american life51\nand a bunch of different places and52\nbasically he has a cool magic trick he54\ncan watch55\na couple fight about something for 1558\nminutes59\nand predict with 85 accuracy whether62\nthey'll be divorced or not63\nuh four years from now if he watche

In [67]:

systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a talk in json format. Your goal is to divide the talk into topics, where each topic ideally would be somewhere 1-5 minutes long. For each topic you should output the following:
1. Name of the topic
2. Very very small description of the topic
3. What second topic starts and what second topic ends 

Output as a json format
"""

mytext = str(res)

In [68]:

resp = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
          "role": "system",
            "content": systemsText
        },
        {
          "role": "user",
            "content": mytext
        }
    ],)

print(resp)

ChatCompletion(id='chatcmpl-8mGUb3uo6pzriEi2DevAUu46B3IWr', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='{\n  "topics": [\n    {\n      "name": "Introduction",\n      "description": "Brief introduction of the talk with a mention of founders\' relationships in startups and their significance.",\n      "start_time": 0,\n      "end_time": 14\n    },\n    {\n      "name": "Marriage Research & Startup Relationships",\n      "description": "Introduction to John Gottman\'s marriage research, the similarities between his findings and startup relationships, and the common topics of disagreement.",\n      "start_time": 15,\n      "end_time": 113\n    },\n    {\n      "name": "Four Major Things to Avoid in Startups",\n      "description": "Discussion of John Gottman\'s Four Horsemen; criticism, contempt, defensiveness, and stonewalling, which can indicate serious trouble in relationships, including those in startups.",\n      "start_time": 114,\n      "end

In [86]:
# print(resp.choices[0].message.content)
topics = json.loads(resp.choices[0].message.content)
topics = topics['topics'][1:]

topics[0]['end_time'] = 153
topics[1]['start_time'] = 153
topics[1]['end_time'] = 262
topics[2]['start_time'] = 262
topics[2]['end_time'] = 483
topics[3]['start_time'] = 483

[{'name': 'Marriage Research & Startup Relationships', 'description': "Introduction to John Gottman's marriage research, the similarities between his findings and startup relationships, and the common topics of disagreement.", 'start_time': 15, 'end_time': 153}, {'name': 'Four Major Things to Avoid in Startups', 'description': "Discussion of John Gottman's Four Horsemen; criticism, contempt, defensiveness, and stonewalling, which can indicate serious trouble in relationships, including those in startups.", 'start_time': 153, 'end_time': 263}, {'name': 'Divide and Conquer in Startups', 'description': 'The significance and method of dividing responsibilities and decisions amongst the founders and team members in a startup, and how this helps in avoiding issues.', 'start_time': 264, 'end_time': 414}, {'name': "Understanding Each Other's Attachment Style", 'description': "The importance of understanding one's and their co-founders' attachment style to ensure better and smoother communicati

In [6]:
# load topics from the file

import pandas as pd
df = pd.read_json('videos-teded.json')



In [67]:
insturctionText = """
I have bunch of topics which consists of a text and a title. I want to determine relationship between them. There might be 4 types of relationship between two topics A and B.
1) A is prerequisite of B - meaning that you have to know A to understand B
2) B is prerequisite of A - meaning that you have to know B to understand A
3) A and B are the same - meaning they are practically the same knowledge it does not matter if you read A or B you will understand the same thing
4) There are no relationship between A and B - meaning to learn A you do not need knowledge from B and to learn B you do not need knowledge from A. 

I will give you 25 topics you determine pairwise the relationship between them.
Give answer as a graph dependency list and return nothing else. If there is no relationship between topics do not output it
"""

systemsText = """
You will be presented with two topics and your job is to determine a relationship between them. There are 5 relationship types between two topics A and B:
1) A is prerequisite of B - meaning that you have to know A to understand B
2) B is prerequisite of A - meaning that you have to know B to understand A
3) A and B are the same - meaning they are practically the same knowledge it does not matter if you read A or B you will understand the same thing
4) A and B are intermingled - meaning there are concepts in A that is needed to understand B and vice versa
5) There are no relationship between A and B - meaning to learn A you do not need knowledge from B and to learn B you do not need knowledge from A
Analize the topics and output ONLY the relationship type as a single number 1,2,3,4 or 5. ONLY OUTPUT SINGLE NUMBER.
"""

topics = """
Topic A:
How does caffeine keep us awake? - Hanan Qasim
Over 100,000 metric tons
of caffeine are consumed around the world every year. That's equivalent to the weight
of 14 Eiffel Towers. Most of this caffeine is consumed
in coffee and tea, but it's also ingested in some sodas, chocolate, caffeine pills, and even beverages labeled decaf. Caffeine helps us feel alert,
focused, happy, and energetic, even if we haven't had enough sleep. But it can also raise our blood pressure,
and make us feel anxious. It's the world most widely used drug. So how does it keep us awake? Caffeine evolved in plants where
it serves a few purposes. In high doses, as it's found in the leaves
and seeds of certain species, it's toxic to insects. But when they consume it in lower doses,
as it's found in nectar, it can actually help them remember
and revisit flowers. In the human body, caffeine acts as a
stimulant for the central nervous system. It keeps us awake by blocking one of
the body's key sleep-inducing molecules, a substance called adenosine. Your body needs a constant supply
of energy, which it gets by breaking down
a high-energy molecule called ATP. In the process, it liberates adenosine,
ATP's chemical backbone. Neurons in your brain have receptors
perfectly tailored to this molecule. When adenosine docks to these receptors, it activates a cascade 
of biochemical reactions that cause neurons to fire more sluggishly and slow the release of important
brain-signaling molecules. In other words, you get sleepy. Caffeine is what's called an adenosine
receptor antagonist. That means it derails this process
of slowing your neurons down by blocking adenosine receptors. Caffeine and adenosine have a similar
molecular structure, close enough that caffeine can wedge into
the adenosine receptors, but not close enough to activate them. To summarize, 
adenosine inhibits your neurons. Caffeine inhibits the inhibitor,
so it stimulates you. Caffeine can also boost positive feelings. In some neurons, the adenosine receptors are linked to receptors 
for another molecule called dopamine. One of dopamine's roles in the brain
is to promote feelings of pleasure. When adenosine docks in one of these
paired receptors, that can make it harder for dopamine
to fit in its own spot, interrupting its mood-lifting work. But when caffeine takes adenosine's place,
it doesn't have the same effect, and dopamine can slide in. There's evidence that caffeine's effects
on adenosine and dopamine receptors can have long-term benefits, too,
reducing the risk of diseases like Parkinson's, Alzheimer's,
and some types of cancer. Caffeine can also ramp up the body's
ability to burn fat. In fact, some sports organizations think that caffeine gives athletes 
an unfair advantage and have placed limits on its consumption. From 1972 until 2004, Olympic athletes had to stay below
a certain blood-caffeine concentration to compete. Of course, not all of caffeine's effects
are so helpful. It might make you feel better
and more alert, but it can also raise your heart rate
and blood pressure, cause increased urination or diarrhea, and contribute to insomnia and anxiety. Plus, the foods and beverages
caffeine is found in have their own impacts on your body
that have to be taken into account. Your brain can adapt to regular
consumption of caffeine. If your adenosine receptors 
are perpetually clogged, your body will manufacture extra ones. That way, even with caffeine around, adenosine can still do its job
of signaling the brain to power down. That's why you may find you need
to consume more and more caffeine to feel as alert. There are more and more adenosine
receptors to block. It's also why if you suddenly 
quit caffeine, you may experience 
an unpleasant withdrawal. With plenty of receptors 
and no competition, adenosine can work overtime, causing symptoms like headaches, tiredness, and depressed moods. But in a few days, the extra adenosine
receptors will disappear, your body will readjust, and you'll feel just as alert as ever, even without an infusion of the world's
most popular stimulant. 


Topic B:
The myth of Prometheus - Iseult Gillespie
Before the creation of humanity, the Greek gods won a great battle against 
a race of giants called the Titans. Most Titans were destroyed or
driven to the eternal hell of Tartarus. But the Titan Prometheus,
whose name means foresight, persuaded his brother Epimetheus
to fight with him on the side of the gods. As thanks, Zeus entrusted the brothers with the task of creating 
all living things. Epimetheus was to distribute the gifts
of the gods among the creatures. To some, he gave flight; to others, the ability 
to move through water or race through grass. He gave the beasts glittering scales, soft fur, and sharp claws. Meanwhile, Prometheus shaped
the first humans out of mud. He formed them in the image of the gods, but Zeus decreed 
they were too remain mortal and worship the inhabitants 
of Mount Olympus from below. Zeus deemed humans subservient
creatures vulnerable to the elements and dependent on the gods for protection. However, Prometheus envisioned his crude
creations with a greater purpose. So when Zeus asked him to decide
how sacrifices would be made, the wily Prometheus planned a trick
that would give humans some advantage. He killed a bull and divided it
into two parts to present to Zeus. On one side, he concealed 
the succulent flesh and skin under the unappealing belly of the animal. On the other, he hid the bones
under a thick layer of fat. When Zeus chose the seemingly
best portion for himself, he was outraged 
at Prometheus's deception. Fuming, Zeus forbade the use
of fire on Earth, whether to cook meat 
or for any other purpose. But Prometheus refused to see
his creations denied this resource. And so, he scaled Mount Olympus
to steal fire from the workshop 
of Hephaestus and Athena. He hid the flames in a hollow fennel stalk
and brought it safely down to the people. This gave them the power to harness
nature for their own benefit and ultimately dominate the natural order. With fire, humans could care 
for themselves with food and warmth. But they could also forge weapons
and wage war. Prometheus's flames acted as a catalyst
for the rapid progression of civilization. When Zeus looked down at this scene,
he realized what had happened. Prometheus had once again
wounded his pride and subverted his authority. Furious, Zeus imposed a brutal punishment. Prometheus was to be chained
to a cliff for eternity. Each day, he would be visited by a vulture
who would tear out his liver and each night his liver would grow back
to be attacked again in the morning. Although Prometheus remained 
in perpetual agony, he never expressed regret 
at his act of rebellion. His resilience in the face of oppression
made him a beloved figure in mythology. He was also celebrated for his mischievous
and inquisitive spirit, and for the knowledge, progress,
and power he brought to human hands. He's also a recurring figure
in art and literature. In Percy Bysshe Shelley's
lyrical drama "Prometheus Unbound," the author imagines Prometheus
as a romantic hero who escapes and continues to spread
empathy and knowledge. Of his protagonist, Shelley wrote, "Prometheus is the type 
of the highest perfection of moral and intellectual nature, impelled by the purest 
and the truest motives to the best and noblest ends." His wife Mary envisaged Prometheus
as a more cautionary figure and subtitled her novel
"Frankenstein: The Modern Prometheus." This suggests the damage of corrupting
the natural order and remains relevant 
to the ethical questions surrounding science and technology today. As hero, rebel, or trickster, Prometheus remains a symbol of our
capacity to capture the powers of nature, and ultimately, he reminds us of the potential 
of individual acts to ignite the world. 

"""

topics3 = """
Start of Topic A:
# Bug in the First Karel Program

Let's revisit our First Karel Program. 

![World before](/images/intro-to-programming-js-en/4_first_program_1.png)

Here's the code that does the job.

```js
move();
pickBeeper();
turnLeft();
move();
turnLeft();
turnLeft();
turnLeft();
move();
move();
putBeeper();
move();
```

Let's run code and make sure it's correct. But what happens if we run it again? A bug!
What happened? We ran the same program, but the world was changed Karel was not on the (1,1) corner anymore and it ran into a wall. 

So you should be aware and run the program for the correct world!
End of Topic A

Start of Topic B:
# History of Bugs in Programs
In the world of computers, a 'bug' is a fancy word for a mistake or problem in a program. Bugs make programs act in ways we didn't expect.

## The First Bug?
A long time ago, in 1947, a real-life moth (yes, the flying kind!) got stuck inside a computer. This caused the machine to go haywire. The engineers who found it joked that they "debugged" the computer by removing the moth. That's the archaic story how the term 'bug' came into the world of computing! But did it, really?

Even before computers looked like the machines we know today, people used the term 'bug' to describe glitches and hitches in electrical systems. Thomas Edison, the famous inventor, even mentioned "bugs" in his writings from the 1870s!

## The Expanding World of Bugs:
As computers evolved, so did bugs. From simple mechanical glitches, bugs grew into complex software issues. They weren't just pesky problems but started influencing big events:

1. **The Ariane 5 Rocket (1996)**: A tiny bug in its software caused this European rocket to explode shortly after its launch. Ouch!

2. **The Y2K Bug**: As the world approached the year 2000, there was a global panic about computers not recognizing '00' as the year 2000 but as 1900. Thankfully, massive efforts debugged the potential mess!
End of Topic B
"""

topics = """
Start of Topic A:
# First Karel Program
Let's write your first Karel program, you know all the commands already. Imagine that Karel lives in the world below

![World before](/images/intro-to-programming-js-en/4_first_program_1.png)

You job is to write program, where Karel will take the beeper on the (2,1) corner, will put it on the (4,2) corner and finish working on (5,2). After program execution world should look like this:

![World after](/images/intro-to-programming-js-en/4_first_program_2.png)

It's easy enough with the following commands:

```js
move
pickBeeper
turnLeft
move
turnLeft
turnLeft
turnLeft
move
move
putBeeper
move
```

But, this is not a program yet. This is the instruction to solve the problem, but it's for humans to understand not for Karel. This is not a program this is an algorithm. Let's turn it to program. To do this we must turn all the commands into the **functions**. Basically **functions** are commands that could be executed. To do that we will just need to add `();` symbols at the end of all the commands. That's it!

So we get:

```js
move();
pickBeeper();
turnLeft();
move();
turnLeft();
turnLeft();
turnLeft();
move();
move();
putBeeper();
move();
```

Yay! It's your first program! Go ahead run it in our [Karel environment](karel_environment_link)


End of Topic A

Start of Topic B:
# `for` Loop in Karel

In some cases we need to repeat some Karel commands several times, We could copy and paste the commands, but it's way better to use the `for` loop.

Generally copy and pasting the code is not a good idea, we end up with the same code in different places. Which makes code changes tricky and error prone.

Let's assume Karel is on the corner of the first street and first avenue - or coordinates (1,1). We want to pick a beeper that is on coordinates(100,1). Pretty straightforward right? We have to write a program with 99 `move();`s and a single `pickBeeper();` at the end. But copying `move();` that many times is quite time-taking and in the process we might miscount add an extra `move();` or miss one. The good news is we can use `for` loop and do it easily.

```js
for (let i = 0; i < some_number; i++) {
    Commands that we want to repeat
}
```

`for` loop consists of 2 parts the header and the body. The header defines number of iterations - or how many times you want to repeat something. And the body defines what you want to repeat - a set of instructions that will be repeated several times. Let's do not dwell on the exact meaning of the `for (let i = 0; i < some_number; i++)` the only thing interesting here is the `some_number` where you can write the actual number - how many times you want to repeat the instructions. And inside the `for` loop or in the body you can write the actual set of instructions.

So the program to solve our stated problem goes as follows:

```js
for (let i = 0; i < 99; i++) {
    move();
}
pickBeeper();
```
If you notices we used tab inside the `for` loop to make clearer for the code reader that `move();` part of the `for` loop's body.

## Several instructions in `for`
Of course we can put several instructions inside the `for` loop. In this case `for` loop executes these set of instructions one by one on each iterations. And the number of iterations is determined in the header of the loop. You can imagine that `for` loop just copies whatever it has inside and pastes it several times. So the code:

```js
for (let i = 0; i < 4; i++) {
    move();
    turnLeft();
}
```

is exactly the same as the code:

```js
move();
turnLeft();
move();
turnLeft();
move();
turnLeft();
move();
turnLeft();
```



End of Topic B
"""


In [68]:
resp = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
          "role": "system",
            "content": systemsText
        },
        {
          "role": "user",
            "content": topics
        }
    ],)

print(resp)

ChatCompletion(id='chatcmpl-8X80qsJKqoWnJIKZhx1EOAWJpBoAE', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='1', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1702906316, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=1131, total_tokens=1132))


In [81]:
import string

def contains_non_ascii(s):
    return any(c not in string.printable for c in s)

non_ascii_rows = df[df['title'].apply(contains_non_ascii)]

longest_title_row = df.loc[df['title'].str.len().idxmax()]

len(longest_title_row['title'])

98

In [20]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a talk. Your goal is to divide the talk into topics, where each topic ideally would be somewhere 1-5 minutes long. For each topic you should output the following:
1. Name of the topic
2. Very very small description of the topic
3. What second topic starts and what second topic ends 
4. Rewrite topic from you own perspective, use simple words
"""

systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a talk. Your goal is to divide the talk into topics, where each topic ideally would be somewhere 1-5 minutes long. For each topic you should output the following:
1. Name of the topic
2. Very very small description of the topic
3. What second topic starts and what second topic ends 

Output as a json format
"""

text = """"""

In [21]:
resp = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
          "role": "system",
            "content": systemsText
        },
        {
          "role": "user",
            "content": text
        }
    ],)

print(resp)

ChatCompletion(id='chatcmpl-8k9BC67vNL51ZIZLQx0C4wzylCgEC', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='[\n{\n  "Topic Name": "Introduction",\n  "Description": "Quick introduction of the talk and its purpose.",\n  "Start Time": 0,\n  "End Time": 12,\n  "Transcript": "you definitely want a co-founder. hey this is michael cyball and dalton. caldwell and welcome to rookie mistakes. we\'ve asked yc founders for their rookie. mistakes so we can share them with you. and help you. avoid these common errors let\'s start"\n},\n{\n  "Topic Name": "Importance of a Written Agreement",\n  "Description": "Speakers express the significance of written agreements in a startup.",\n  "Start Time": 15,\n  "End Time": 47,\n  "Transcript": "with our first anonymous story from yc. founders and mistakes to avoid at the. beginning when everything\'s going well. you don\'t know how you\'re going to. handle disagreements or bad actions by. the other person it\'s extremel

In [37]:
import json

a = json.loads(resp.choices[0].message.content)
a[6]

{'Topic Name': 'How should the CEO set up a company?',
 'Description': 'Advice on how the CEO should set up a company and manage equity split to avoid issues.',
 'Start Time': 259,
 'End Time': 320,
 'Transcript': "how should the ceo set up a company so. that if this battle. you know if there is a co-founder fight. it's it's not fatal to the company the. closest to equal you can get the better. because it avoids this oh this person. has 10 percent more so it's their. company you see all this drama that. happens over equity split so. equal is good however here's a pro tip. for you. a straight 50 50 deadlock. is rough and we see that kill companies. a lot and so it might be reasonable. for the ceo to have one extra share. so you effectively have the same. ownership your equal co-founders but. that one extra share is you agreeing in. writing early that in the event of a. 50-50 deadlock. there's a tiebreaker vote dalton before. we close."}